# Calculating the PDO

#### The data comes from the 20th Century Reanalysis (Version 3).
- Data spans from 01-01-1836 to 12-31-2015.
- Skin temperture is used along with a land sea mask to aquire just sea surface temperature.

Data can be aquired from the PSL Thredds server:
- https://psl.noaa.gov/thredds/catalog/Datasets/20thC_ReanV3/catalog.html

In [1]:
import xarray as xr
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import cartopy
from eofs.standard import Eof

#### The land sea mask was manipulated to have NaN values for land and 1 for water. The mask is then multiplied by skin temperature to get values just over water.

In [2]:
# Had to switch the values to make land NaN and water 1
mask = xr.open_dataset('/home/scratch/20CR_v3/mask.nc')

In [3]:
%%time
# The data was combined into one file for all years. Mean and standard deviation were alos calculated.
dailyskt = xr.open_dataset('/home/scratch/20CR_v3/daily_skt_1836_2015.nc')*mask.land
meanskt = xr.open_dataset('/home/scratch/20CR_v3/climos/skint_mean_dayofyear_1836_2015.nc')*mask.land
stdevskt = xr.open_dataset('/home/scratch/20CR_v3/climos/skint_stdev_dayofyear_1836_2015.nc')*mask.land

CPU times: user 42.1 s, sys: 1min 22s, total: 2min 4s
Wall time: 2min 4s


#### Slicing to the core region in the Pacific

In [4]:
# Slice the data to the core region
dailyskt = dailyskt.sel(lat=slice(20, 70), lon=slice(110, 260))
meanskt = meanskt.sel(lat=slice(20, 70), lon=slice(110, 260))
stdevskt = stdevskt.sel(lat=slice(20, 70), lon=slice(110, 260))

#### Standardized anomaly calculation

In [5]:
%%time

stdanompdo = []

# Calculate the standardized anomalies and append them to a list
for i in dailyskt.time.values:
   
    current = dailyskt.skt.sel(time=str(i)).mean(dim='lat').mean(dim='lon')
    current_day = dt.datetime.strptime(str(current.time.values)[0:10], '%Y-%m-%d')
    dayofyear = int(current_day.strftime('%j'))
   
    # Select the daily mean and daily standard deviation of skt
    dailyskt_mn = meanskt.sel(dayofyear = dayofyear).mean(dim='lat').mean(dim='lon')
    dailyskt_std = stdevskt.sel(dayofyear = dayofyear).mean(dim='lat').mean(dim='lon')
   
    # Calculate the standardized anomaly of skt and add to the list
    stdanom = (current - dailyskt_mn.skt) / dailyskt_std.skt
    stdanompdo.append(stdanom)

# Add the standardized anomalies to the existing file
dailyskt['pdostdanom']=('time', stdanompdo)

CPU times: user 7min 21s, sys: 10.4 s, total: 7min 32s
Wall time: 7min 32s


#### Save the index to its own file

In [6]:
dailyskt.pdostdanom.to_netcdf(path='daily_PDO_1836_2015.nc')
dailyskt.pdostdanom.to_netcdf(path='/home/scratch/20CR_v3/daily_PDO_1836_2015.nc')